In [64]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import scipy.optimize as opt
from tabulate import tabulate
from scipy.stats import laplace, uniform
import math

## Постановка задачи:
Сгенерировать выборку объёмом $100$ элементов для нормального распределения $N(x,0,1).$ 

По сгенерированной выборке оценить параметры $\mu$ и $\sigma$ нормального закона методом максимального правдоподобия. В качестве основной гипотезы $H_{0}$ будем считать, что сгенерированное распределение имеет вид $N(x,\hat{\mu}, \hat{\sigma})$. Проверить основную гипотезу, используя критерий согласия $\chi^{2}$. В качестве уровня значимости взять $\alpha = 0.05$. Привести таблицу вычислений $\chi^{2}$. 


Сгенерировать выборку объемом $20$ элементов для распределения Лапласа. Проверить гипотезу $H_0^*$ нормальности этой выборки, используя критерий согласия $\chi^{2}$.
	


In [130]:
points = 20
sample_size_normal = 100
sample_size_laplace = sample_size_uniform = 20
start, end = -1.5, 1.5
alpha = 0.05
p_ = 1 - alpha

mu_, sigma_squared = 0, 1


def find_k(size):
    return math.ceil(1.72 * (size) ** (1/3))

## Метод максимального правдоподобия

In [131]:
def max_likelihood_estimation(sample):
    mu = np.mean(sample)
    sigma = np.std(sample)
    print("mu = ", np.around(mu, decimals=2),
          " sigma=", np.around(sigma, decimals=2))
    return mu, sigma

## Критерий согласия $\chi^{2}$

In [139]:
def calculate_chi2(p,n,sample_size):
    tmp = np.multiply((n - sample_size * p), (n - sample_size * p))
    chi2 = np.divide(tmp, p * sample_size)
    return chi2

def is_hypo_accepted(quantile, chi2):
    if quantile > np.sum(chi2):
        return True
    return False

def find_all_probabilities(borders, hypothesis, sample):
    p = np.array(hypothesis(start))
    n = np.array(len(sample[sample < start]))

    for i in range(k - 2):
        p_i = hypothesis(borders[i + 1]) - hypothesis(borders[i])
        p = np.append(p, p_i)
        n_i = len(sample[(sample < borders[i + 1]) & (sample >= borders[i])])
        n = np.append(n, n_i)

    p = np.append(p, 1 - hypothesis(end))
    n = np.append(n, len(sample[sample >= end]))
    
    return p,n

    
def chi_square_criterion(sample, mu, sigma):
    hypothesis = lambda x: stats.norm.cdf(x, loc=mu, scale=sigma)
    borders = np.linspace(start, end, num=k - 1)
    p,n = find_all_probabilities(borders, hypothesis, sample)
    chi2 = calculate_chi2(p,n, len(sample))
    quantile = stats.chi2.ppf(p_, k - 1)
    isAccepted = is_hypo_accepted(quantile, chi2)
    return chi2, isAccepted, borders, p, n

## Вывод результатов в таблицу

In [154]:
def build_table(chi2, borders, p, n, sample_size):
    rows = []
    headers = ["$i$", "$\\Delta_i = [a_{i-1}, a_i)$", "$n_i$", "$p_i$",
               "$np_i$", "$n_i - np_i$", "$(n_i - np_i)^2/np_i$"]   
    for i in range(0, len(n)):
        if i == 0:
            limits = ["$-\infty$", np.around(borders[0], decimals=2)]
        elif i == len(n) - 1:
            limits = [np.around(borders[-1], decimals=2), "$\infty$"]
        else:
            limits = [np.around(borders[i - 1], decimals=2), np.around(borders[i], decimals=2)]
        rows.append([i + 1, limits, n[i],
             np.around(p[i], decimals=4),
             np.around(p[i] * sample_size, decimals=2),
             np.around(n[i] - sample_size * p[i], decimals=2),
             np.around(chi2[i], decimals=2)] )
    rows.append(["\\sum", "--", np.sum(n), np.around(np.sum(p), decimals=4),
                 np.around(np.sum(p * sample_size), decimals=2),
                 -np.around(np.sum(n - sample_size * p), decimals=2),
                 np.around(np.sum(chi2), decimals=2)]
    )
    return tabulate(rows, headers, tablefmt="latex_raw")


In [155]:
def checkAcception(isAccepted):
    if isAccepted:
        print("\nHypothesis Accepted!")
    else:
        print("\nHypothesis Not Accepted!")

### Нормальное распределение

In [156]:
k = find_k(sample_size_normal)
normal_sample = np.random.normal(0, 1 , sample_size_normal)
mu, sigma = max_likelihood_estimation(normal_sample)
chi2, isAccepted, borders, p, n = chi_square_criterion(normal_sample, mu, sigma)
print(build_table(chi2, borders, p, n, 100))
checkAcception(isAccepted)

mu =  0.04  sigma= 0.86
\begin{tabular}{llrrrrr}
\hline
 $i$   & $\Delta_i = [a_{i-1}, a_i)$   &   $n_i$ &   $p_i$ &   $np_i$ &   $n_i - np_i$ &   $(n_i - np_i)^2/np_i$ \\
\hline
 1     & ['$-\\infty$', -1.5]          &       4 &  0.0364 &     3.64 &           0.36 &                    0.04 \\
 2     & [-1.5, -1.0]                  &       7 &  0.0764 &     7.64 &          -0.64 &                    0.05 \\
 3     & [-1.0, -0.5]                  &      12 &  0.1518 &    15.18 &          -3.18 &                    0.67 \\
 4     & [-0.5, 0.0]                   &      27 &  0.2168 &    21.68 &           5.32 &                    1.31 \\
 5     & [0.0, 0.5]                    &      25 &  0.2226 &    22.26 &           2.74 &                    0.34 \\
 6     & [0.5, 1.0]                    &      14 &  0.1643 &    16.43 &          -2.43 &                    0.36 \\
 7     & [1.0, 1.5]                    &       6 &  0.0872 &     8.72 &          -2.72 &                    0.85 \\
 8     & 

### Распределение Лапласа

In [157]:
k = find_k(sample_size_laplace)
laplace_sample = distribution = laplace.rvs(size=20, scale=1 / math.sqrt(2), loc=0)
mu, sigma = max_likelihood_estimation(laplace_sample)
chi2, isAccepted, borders, p, n = chi_square_criterion(laplace_sample, mu, sigma)
print(build_table(chi2, borders, p, n,20 ))
checkAcception(isAccepted)

mu =  -0.21  sigma= 1.01
\begin{tabular}{llrrrrr}
\hline
 $i$   & $\Delta_i = [a_{i-1}, a_i)$   &   $n_i$ &   $p_i$ &   $np_i$ &   $n_i - np_i$ &   $(n_i - np_i)^2/np_i$ \\
\hline
 1     & ['$-\\infty$', -1.5]          &       2 &  0.101  &     2.02 &          -0.02 &                    0    \\
 2     & [-1.5, -0.5]                  &       5 &  0.2854 &     5.71 &          -0.71 &                    0.09 \\
 3     & [-0.5, 0.5]                   &       8 &  0.3711 &     7.42 &           0.58 &                    0.04 \\
 4     & [0.5, 1.5]                    &       5 &  0.1965 &     3.93 &           1.07 &                    0.29 \\
 5     & [1.5, '$\\infty$']            &       0 &  0.0459 &     0.92 &          -0.92 &                    0.92 \\
 \sum  & --                            &      20 &  1      &    20    &          -0    &                    1.34 \\
\hline
\end{tabular}

Hypothesis Accepted!


### Равномерное распределение

In [158]:
k = find_k(sample_size_uniform)
uniform_sample = distribution =  uniform.rvs(size=20, loc=-math.sqrt(3), scale=2 * math.sqrt(3))
mu, sigma = max_likelihood_estimation(uniform_sample)
chi2, isAccepted, borders, p, n = chi_square_criterion(uniform_sample, mu, sigma)
print(build_table(chi2, borders, p, n, 20))
checkAcception(isAccepted)

mu =  -0.0  sigma= 0.97
\begin{tabular}{llrrrrr}
\hline
 $i$   & $\Delta_i = [a_{i-1}, a_i)$   &   $n_i$ &   $p_i$ &   $np_i$ &   $n_i - np_i$ &   $(n_i - np_i)^2/np_i$ \\
\hline
 1     & ['$-\\infty$', -1.5]          &       0 &  0.062  &     1.24 &          -1.24 &                    1.24 \\
 2     & [-1.5, -0.5]                  &       6 &  0.2421 &     4.84 &           1.16 &                    0.28 \\
 3     & [-0.5, 0.5]                   &       9 &  0.392  &     7.84 &           1.16 &                    0.17 \\
 4     & [0.5, 1.5]                    &       4 &  0.242  &     4.84 &          -0.84 &                    0.15 \\
 5     & [1.5, '$\\infty$']            &       1 &  0.0619 &     1.24 &          -0.24 &                    0.05 \\
 \sum  & --                            &      20 &  1      &    20    &           0    &                    1.88 \\
\hline
\end{tabular}

Hypothesis Accepted!
